## Workspace Management demo

This notebook presents examples of how to manage user workspaces.

In [ ]:
import requests
import datetime

In [ ]:
base_domain = "demo.eoepca.org"
# base_domain = "develop.eoepca.org"
workspaces_endpoint = f'https://workspace-api-open.{base_domain}/workspaces'

### Creation and retrieval of credentials

The following request will create a user workspace.

In [ ]:
preferred_workspace_name = f"my-new-user-{datetime.datetime.now().timestamp():.0f}"
preferred_workspace_name

In [ ]:
response = requests.post(
    workspaces_endpoint,
    json={
        "preferred_name": preferred_workspace_name,
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
response.json()

Workspace creation is an asynchronous process, so when we try to retrieve the workspace data right now, we see that it is still provisioning.

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()
response.json()

In [ ]:
import time
# wait for provisioning
time.sleep(10)

Now all the credentials should be set up and are available:

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()

workspace_data = response.json()
workspace_data["storage"]["credentials"]['secret'] = "hidden_for_demo"
workspace_data["container_registry"]["password"] = "hidden_for_demo"
workspace_data

## Redeploy

In case something went wrong or an update is necessary, it's possible to trigger the provisioning again:

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/redeploy",
)
response.raise_for_status()
response

## Deletion
Workspaces can also be deleted like this:

In [ ]:
response = requests.delete(
    f"{workspaces_endpoint}/{actual_workspace_name}",
)
response.raise_for_status()
response